# T-Bot Dashboard

Welcome to TBot

In [1]:
# imports
import pandas as pd
import panel as pn
pn.extension('plotly')
import plotly.express as px
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

ImportError: cannot import name 'registry' from 'pandas.core.dtypes.base' (C:\Users\visha\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\dtypes\base.py)

In [ ]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [ ]:
# Import the CSVs to Pandas DataFrames
file_path = Path("../Data/toronto_neighbourhoods_census_data.csv")
census_data = pd.read_csv(file_path, index_col="year")

file_path = Path("../Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

### Global available data

In [ ]:
# Getting the data from the top 10 expensive neighbourhoods
top10_nbhood=pd.DataFrame()
census_2016=census_data.filter(like='2016', axis=0)
top10=census_2016.sort_values('average_house_value',ascending=False).nlargest(10,'average_house_value')
top10_2016nbhood=top10.loc[:,['neighbourhood','average_house_value']]
top10_2016nbhood['average_house_value'] =top10_2016nbhood['average_house_value'].apply(lambda row: "${:.3f}MM".format((row/1000000)))
top10_2016nbhood

In [ ]:
# Calculate the mean number of dwelling types units per year
# YOUR CODE HERE!
#make_float = lambda row: "${:,.2f}".format(row)
num_dwtypes_bynbhood=pd.DataFrame()
num_dwtypes_bynbhood=census_data.loc[:,:'other_house']
mean_dwtypes_peryr=num_dwtypes_bynbhood.groupby('year').sum()
#mean_dwtypes_peryr.apply(make_float)
mean_dwtypes_peryr.head()

In [ ]:
# Calculate the average monthly shelter costs for owned and rented dwellings
# YOUR CODE HERE!


### Panel Visualization Functions

In [ ]:
# Define Panel visualization functions
def neighbourhood_map(df,ltd,lng,sz,clr):
    """Neighbourhood Map"""
    nbmap = px.scatter_mapbox(df,
                            lat=ltd,
                            lon=lng,
                            size=sz,
                            color=clr,
                            zoom=9,width=1000,height=450)
    return nbmap

In [ ]:
#Bar chart function - no dropdown
def create_bar_chart(data,xaxis,yaxis,rt,ht,wd):
    plt=data.hvplot.bar(x=xaxis,y=yaxis,rot=rt,height=ht,width=wd)
    return plt

In [ ]:
#Bar with dropdown
def create_bar_chart2(data,xaxis,grp,rt,ht,wd):
    plt=data.hvplot.bar(x=xaxis,groupby=grp,legend='top_left',rot=rt,height=ht,width=wd)
    return plt

In [ ]:
#Line no dropdown
def create_line_chart(data,clr,bgbg,ht,wd,rt):
    """
    Create a line chart based in the data argument.
    """
    plt2=data.hvplot.line(color=clr,height=ht,width=wd,rot=rt,yformatter='%f').opts(bgcolor=bgbg)
    return plt2

In [ ]:
#Helper function that calculates Avg. House Value per year
def average_house_value(data,grp,value_needing_avg):
    """Average house values per year."""
    
    avg_houseval_peryear=round(data.groupby(grp)[value_needing_avg].mean(),2)
    return avg_houseval_peryear

In [ ]:
#Helper function that calculates Avg. House Value by neighbourhood
def average_value_by_neighbourhood(data,col1,col2):
    """Average house values by neighbourhood."""
    avg_hsval_bynbhood=pd.DataFrame()
    avg_hsval_bynbhood=data[[col1,col2]]
    return avg_hsval_bynbhood

In [ ]:
#Helper function that calculates Avg. House Value per year
def number_dwelling_types(data2,end_col,grp):
    """Number of dwelling types per year"""
    num_dwtypes_bynbhood=pd.DataFrame()
    num_dwtypes_bynbhood=data2.loc[:,:end_col]
    mean_dwtypes_peryr=num_dwtypes_bynbhood.groupby(grp).sum()
    return mean_dwtypes_peryr

In [ ]:
#Helper function that calculates Average house value for all Toronto's neighbourhoods per year
def average_house_value_snapshot(data,col1,col2,desired_year):
    """Average house value for all Toronto's neighbourhoods per year"""
    avg_hsval_bynbhood=pd.DataFrame()
    avg_hsval_bynbhood=data[[col1,col2]]
    avg_hsval_bynbhood=avg_hsval_bynbhood.loc[desired_year]
    return avg_hsval_bynbhood

In [ ]:
#Helper function that calculates most expensive neighbourhoods
def top_most_expensive_neighbourhoods(data,col,col1,col2):
    """Top 10 most expensive neighbourhoods."""
    top10_nbhood=pd.DataFrame()
#     census_2016=data.filter(like=filterby, axis=0)
    top10_nbhood=data.nlargest(10,col)
    top10_nbhood=top10.loc[:,[col1,col2]]
    #top10_2016nbhood['average_house_value'] =top10_2016nbhood['average_house_value'].apply(lambda row: "${:.3f}MM".format((row/1000000)))
    return top10_nbhood

In [ ]:
#Sunburst chart for cost analysis
def sunburts_cost_analysis(data,p1,p2,val,clr):
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    snbrst = px.sunburst(data,path=[p1,p2],values=val,color=clr)
    return snbrst

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
mostexp=top_most_expensive_neighbourhoods(census_data,'average_house_value','neighbourhood','average_house_value')
barplt= create_bar_chart(mostexp,'neighbourhood','average_house_value',90,700,700)
pane1 = pn.panel(barplt)

In [ ]:
ndt= number_dwelling_types(census_data,'other_house','year')
lineplt1= create_line_chart(ndt.iloc[0],'black','lightblue',700,900,90)
lineplt2= create_line_chart(ndt.iloc[1],'black','lightblue',700,900,90)
lineplt3= create_line_chart(ndt.iloc[2],'black','lightblue',700,900,90)
lineplt4= create_line_chart(ndt.iloc[3],'black','lightblue',700,900,90)

pane2 = pn.panel(lineplt1)
pane3 = pn.panel(lineplt2)
pane4 = pn.panel(lineplt3)
pane5 = pn.panel(lineplt4)

In [ ]:
panelrow= pn.Row(pane1,pane2,pane3,pane4,pane5)

# Define a welcome text
# YOUR CODE HERE!

# Create a tab layout for the dashboard
# YOUR CODE HERE!
Tbot_tabs = pn.Tabs(
    ("Toronto's Real Estate Dashboard",panelrow),
    ("Top most expensive neighbourhoods", pane1),
    ("2001 Number of Dwelling Types",pane2),
    ("2006 Number of Dwelling Types",pane3),
    ("2011 Number of Dwelling Types",pane4),
    ("2016 Number of Dwelling Types",pane5))
# Tbot_tabs

## Serve the Panel Dashboard

In [ ]:
Tbot_tabs.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# neighbourhood_map().show()

In [ ]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [ ]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [ ]:
# average_house_value()

In [ ]:
# average_value_by_neighbourhood()

In [ ]:
# number_dwelling_types()

In [ ]:
# average_house_value_snapshot()

In [ ]:
# top_most_expensive_neighbourhoods()

In [ ]:
# sunburts_cost_analysis()